In [1]:
import pandas as pd
import urllib.request
import csv
import matplotlib.pyplot as plt
import  platform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup

## User Input and Crawling
<br>
1. 사용자에게 가수명, 곡명 입력 받기
<br>
2. 사용자가 입력한 곡의 가사 크롤링

In [2]:
while True :
    print("가수명-곡명 형태로 입력 : ",end='')
    target_singer, target_title = input().split('-')
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' +urllib.parse.quote_plus(target_singer +' '+target_title + ' 곡정보')
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    try :
        target_album = soup.select('.info.txt_4 > .info_group')[1].text.split()[1]
        target_genre = soup.select('.info.txt_4 > .info_group')[3].text.strip().split()[1]
        target_days = soup.select('.info.txt_4 > .info_group')[2].text.strip().split()[1].replace('.','')
        season = soup.select('.info.txt_4 > .info_group')[2].text.strip().split('.')[1]
        if '03' <= season and season <= '05':
            target_season = '봄'
        elif '06' <= season and season <= '08':
            target_season = '여름'
        elif '09' <= season and season <= '11':
            target_season = '가을'
        else:
            target_season = '겨울'
        target_lyric = soup.select('.text.no_ellipsis.type_center._content_text')[0].get_text()
        break
        
    # 곡이 없거나 잘못 입력된 경우    
    except :
        print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요")
        pass
    
print("장르 : " + target_genre + "\n" + "발매계절 : " + target_season)

target_title_org = target_title.strip()
target_singer_org = target_singer.strip()

target_singer = target_singer.replace(' ','')
target_title  = target_title.replace(' ','') 
target_genre = target_genre.strip()
target_season = target_season.strip()
target_lyric = target_lyric.strip()
 

가수명-곡명 형태로 입력 : 조관우-겨울이야기
장르 : 발라드(국내)
발매계절 : 겨울


## csv에서 입력된 곡 정보찾기 

In [66]:
 while True :
    print("가수명-곡명 형태로 입력 : ",end='')
    target_singer, target_title = input().split('-')
      
    df_and = data[(data['가수'] == target_singer) & (data['제목'] == target_title)]
      
    len(df_and.index)     
         
    try : 
        if len(df_and.index) > 0 :

            indices = pd.Series(data.index,index= [ data['제목'], data['가수']]  ).drop_duplicates() 

            idx = indices[(target_title, target_singer)][0] 

            target_album = data['앨범명'].iloc[idx] 
            target_genre = data['장르'].iloc[idx] 
            target_days  = data['발매일'].iloc[idx] 
            target_season = data['계절'].iloc[idx]  
          

        else: 
            print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요")
            pass
        break
        
    # 곡이 없거나 잘못 입력된 경우    
    except :
        print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요")
        pass
    
 
 print("가수: " + target_singer + "\n" + "제목: " + target_title  + "\n" +  "앨범: " + target_album + "\n"  + "장르: " + target_genre + "\n" + "발매계절: " + target_season + "\n" + "발매일: " + str(target_days))


target_title_org  = target_title.strip()
target_singer_org = target_singer.strip()

target_singer = target_singer.replace(' ','')
target_title  = target_title.replace(' ','') 
target_genre = target_genre.strip()
target_season = target_season.strip() 
  
df_and.head() 
 

가수명-곡명 형태로 입력 : 조관우-겨울이야기
가수: 조관우
제목: 겨울이야기
앨범: 윤민수(Vibe)의 겨울이야기
장르: 발라드
발매계절: 겨울
발매일: 20061222


C:\Users\eomtm\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2894: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


,제목,가수,장르,발매일,앨범명,가사,계절
27803,겨울이야기,조관우,발라드,20061222,윤민수(Vibe)의 겨울이야기,내겐 잊혀지지 않는 겨울얘기가 있어 \n그 얘기속엔 두여인이 나오고 \n추억의 노래...,겨울


## data load
<br>
1.약 42,000개의 크롤링 된 데이터 셋 읽어오기
<BR>
2.사용자가 입력한 곡이 있는지 검사
    <BR>
3.없다면 추가 후 새로 data load

In [24]:
stop = 0
while True:
    data = pd.read_csv('가사전처리수정본.csv', low_memory=False)
    if stop == 1 :
        break
    
    # 사용자가 입력한 곳이 데이터 셋에 있는지 확인
    for i in range(len(data[["제목","가수"]])) :
        temp_title = str(data.loc[i][0]).replace(' ','')
        temp_singer = str(data.loc[i][1]).replace(' ','')
        if (target_title in temp_title) and (target_singer in temp_singer) :
            target_genre = data.loc[i][2]
            target_season = data.loc[i][6]
            stop = 1
            break
    
    # 없다면, 입력한 곡 추가
    if stop == 0 :
        temp = [target_title,target_singer,target_genre,target_days,target_album,target_lyric,target_season]
        with open('가사전처리수정본.csv','a',encoding='utf-8-sig',newline='') as f:
            Writer = csv.writer(f)
            Writer.writerow(temp)
        stop = 1

In [35]:
if data['가사'].isnull().sum() :
    data['가사'] = data['가사'].fillna('')
else :
    print("결측값 없음")

결측값 없음


## data 중복찾기
<br>
1.제목과 가수로 중복된 데이터 검색
<BR>
2.중복된 곡의 정보 조회

In [ ]:
df = pd.DataFrame(data, columns=['제목','가수']) 
df2 = df[df.duplicated(['제목','가수'], keep = False)]
df_sorted_by_values = df2.sort_values(by='제목' ,ascending=False)
df_sorted_by_values

In [6]:
is_title_dup = data['제목'] == '힘을 내'
title_dup = data[is_title_dup]
title_dup

,제목,가수,장르,발매일,앨범명,가사,계절
29440,힘을 내,주나목,CCM,20020220,주님 여기 계심으로,"\n힘을 내\n너 지쳐 쓰러져\n힘들때 주님 너를 지키시니,\n\n힘을 내\n힘을 ...",겨울
35785,힘을 내,주나목,CCM,20130401,힘을 내,"\n힘을 내\n너 지쳐 쓰러져\n힘들때 주님 너를 지키시니,\n\n힘을 내\n힘을 ...",봄


In [7]:
data.index 

RangeIndex(start=0, stop=42052, step=1)

In [ ]:
#data = data.head(20000) 
print(target_title_org)
print(target_singer_org)

In [10]:
# ngram_range = (1,2) 단어의 묶음을 1개부터 2개까지 설정
tfidf = TfidfVectorizer(stop_words='english',analyzer = 'word',  min_df=2, ngram_range = (2, 4), sublinear_tf=True)

# 가사에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['가사'])
print(tfidf_matrix.shape)

tfidf.fit(data['가사']) # 벡터라이저가 단어들을 학습합니다.
sorted(tfidf.vocabulary_.items()) # 단어사전을 정렬합니다. 
#tfidf.vocabulary_ # 벡터라이저가 학습한 단어사전을 출력합니다. 

(42052, 858822)


[('02 18', 0),
 ('0을 향해', 1),
 ('0을 향해 달려가', 2),
 ('0을 향해 달려가 이상', 3),
 ('10 billion', 4),
 ('10 billion save', 5),
 ('10 billion save 닮은', 6),
 ('10 minute', 7),
 ('10 십년이', 8),
 ('10 십년이 가도', 9),
 ('10 십년이 가도 너를', 10),
 ('10 하나', 11),
 ('10 하나 다섯여섯', 12),
 ('10 하나 다섯여섯 일곱', 13),
 ('1000 years', 14),
 ('1000 years 그댈', 15),
 ('1000 years 그댈 기다렸어', 16),
 ('1000 years 때론', 17),
 ('1000 years 때론 함께하며', 18),
 ('1000번의 이름을', 19),
 ('1000번의 이름을 불렀어', 20),
 ('1000번의 이름을 불렀어 1000개의', 21),
 ('1000일 동안', 22),
 ('1000일 동안 매일', 23),
 ('1000일 동안 매일 너의', 24),
 ('100점 만점에', 25),
 ('100프로 동의해', 26),
 ('100프로 동의해 너와의', 27),
 ('100프로 동의해 너와의 이별이', 28),
 ('100프로 한가한', 29),
 ('100프로 한가한 남자가', 30),
 ('100프로 한가한 남자가 돼줄게', 31),
 ('10cm씩 멀어져', 32),
 ('10cm씩 멀어져 가다', 33),
 ('10cm씩 멀어져 가다 가끔씩은', 34),
 ('10개월 매일', 35),
 ('10개월 매일 보는', 36),
 ('10개월 매일 보는 얼굴', 37),
 ('10년 뒤에', 38),
 ('10년 만에', 39),
 ('10년 전부터', 40),
 ('10년 전에', 41),
 ('10년 전으로', 42),
 ('10년이 가도', 43),
 ('10년이 가도 자리에', 44),
 ('10년이 가도 자리에 음악이', 45

## 코사인 유사도 실행

In [11]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [12]:
indices = pd.Series(data.index,index= [ data['제목'], data['가수']]  ).drop_duplicates() 
#print(len(data))
#print(indices.index)    
#print(indices.head())
idx = indices[(target_title_org,target_singer_org)][0]
print(idx)

27803


C:\Users\eomtm\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2894: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


## 유사곡 추천
1.사용자가 입력한 곡을 기준으로 장르, 발매계절이 같은 순으로 출력
<br>
2.장르가 같은것이 없다면, 발매계절이 같은 순으로 출력

In [1]:
def get_recommendations(title, singer, genre, season, cosine_sim=cosine_sim):
    # 선택한 음악의 가수, 제목으로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 음악를 가지고 연산할 수 있습니다.
    #idx = indices[title]
    idx = indices[(title, singer)][0]
   
    # 모든 음악에 대해서 해당 음악과의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))
 
    # 유사도에 따라 음악들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 1000개의 음악을 받아옵니다.
    sim_scores = sim_scores[1:1001]

    #코사인값은 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖음
    #print(sim_scores)
    
    # 가장 유사한 1000개의 음악의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 곡 출력
    num = 0
    for i in range(len(movie_indices)) :
        if num == 10 :
            break
        if float == type(data['장르'].iloc[movie_indices[i]]):
            continue
        if (data['장르'].iloc[movie_indices[i]] not in genre) :
            continue
        if(data['계절'].iloc[movie_indices[i]] != season) :
            continue
        num += 1
        a = data['가수'].iloc[movie_indices[i]] 
        b = data['제목'].iloc[movie_indices[i]]
        c = data['장르'].iloc[movie_indices[i]]
        d = data['계절'].iloc[movie_indices[i]]
        print(a+'-'+b+' ('+c+'), '+d+'발매') 
    
    if num < 10 :
        for i in range(len(movie_indices)) :
            if num == 10 :
                break
            if (data['계절'].iloc[movie_indices[i]] != season) :
                continue
            num += 1
            a = data['가수'].iloc[movie_indices[i]] 
            b = data['제목'].iloc[movie_indices[i]]
            c = data['장르'].iloc[movie_indices[i]]
            d = data['계절'].iloc[movie_indices[i]]
            print(a+'-'+b+' ('+c+'), '+d+'발매') 

SyntaxError: invalid syntax (<ipython-input-1-c1e180805b78>, line 40)

In [64]:
print(target_title_org )
print(target_title)
print(target_singer_org )
print(target_genre )
print(target_season)


겨울 한 끼
겨울한끼
안수지
발라드
겨울


In [14]:
get_recommendations(target_title_org, target_singer_org, target_genre, target_season)

C:\Users\eomtm\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2940: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


노을-어떤말도 (발라드), 겨울발매
김주리-기억하나요 (발라드), 겨울발매
비비-그날 이후에 (발라드), 겨울발매
쥬비-I Love You Song (발라드), 겨울발매
루다-떠나며 그대에게... Goodbye (발라드), 겨울발매
허각-나를 사랑했던 사람아 (발라드), 겨울발매
김정민-바람이 하는 말 (발라드), 겨울발매
안수지-겨울 한 끼 (발라드), 겨울발매
조영언-오늘의 날씨 (나오지 말 걸 그랬어) (발라드), 겨울발매
바이브-Vibe... Real Life... (발라드), 겨울발매


전체 길이를 나누어 진행
list에 넣으면서 유사도 같이 넣기
sort 후 앞에서 10개 출력

## ngram_range = 2~4로 선정
<br>
아래 nrgam_range에 따른 결과를 보면, 1~2를 제외한 나머지의 결과는 동일
<br>
가사의 경우 1개의 단어보다 2개의 단어가 의미를 갖는 경우가 많다고 생각

1~2 : 19997,210240

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
바나나 보트-부비 Party (댄스), 가을발매
슈퍼걸-토요일 밤에 (댄스), 가을발매
투포케이-빨리와 (댄스), 가을발매
장우혁-Scream (댄스), 가을발매
원더보이즈-신난다 신나 (댄스), 가을발매
손호영-해피 바이러스 (댄스), 가을발매
탑독-비가 와서 그래 (댄스), 가을발매

1~4 : 19997,360352

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매

1~5 : 19997,425464

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매


2~4 : 19997,294707

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매

2~5 : 19997,359819

천명훈-정글 (댄스), 가을발매
크리스티나-Pay Day (댄스), 가을발매
엔소닉-Run & Run (댄스), 가을발매
거북이-사계 (댄스), 가을발매
빅스타-왜이래 (댄스), 가을발매
김완선, 용준형-Be Quiet (댄스), 가을발매
양파-뱀파이어 (댄스), 가을발매
유승준-Paradise (댄스), 가을발매
별-비밀하나 (댄스), 가을발매
인디고-그 누구보다 더 (댄스), 가을발매